# Zero-shot mutant prediction For T7

In [3]:
from transformers import AutoTokenizer, AutoModelForMaskedLM
import torch
import pandas as pd
from Bio import SeqIO
from tqdm.notebook import tqdm
import warnings
warnings.filterwarnings("ignore")
import os
os.environ["http_proxy"] = "http://g5:15777"
os.environ["https_proxy"] = "http://g5:15777"

def read_seq(seq_file):
    for record in SeqIO.parse(seq_file, "fasta"):
        return str(record.seq)

wild_type = f"../example_data/expr/t7.fasta"
sequence = read_seq(wild_type)

all_mutants = []
for i, wt in enumerate(sequence):
    for aa in "ACDEFGHIKLMNPQRSTVWY":
        if aa != wt:
            mutant = f"{wt}{i+1}{aa}"
            all_mutants.append(mutant)
df = pd.DataFrame({"mutant": all_mutants})

In [7]:
base_model_path = "AI4Protein/t7_model_"

for i in range(0, 5):
    model_path = f"{base_model_path}{i}"
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = AutoModelForMaskedLM.from_pretrained(model_path, trust_remote_code=True)
    tokenizer = AutoTokenizer.from_pretrained("AI4Protein/Prime_690M", trust_remote_code=True)
    model.eval()
    model = model.to(device)

    tokenized = tokenizer(sequence, return_tensors="pt").to(device)
    input_ids = tokenized["input_ids"]
    attention_mask = tokenized["attention_mask"]

    with torch.no_grad():
        output = model(input_ids, attention_mask=attention_mask)
        logits = output.logits[0, 1:-1, :].log_softmax(dim=-1)
    
    scores = []
    for mutant in tqdm(df["mutant"]):
        score = 0
        for sub_mutant in mutant.split(":"):
            wt, idx, mt = sub_mutant[0], int(sub_mutant[1:-1]) - 1, sub_mutant[-1]
            score += (logits[idx, tokenizer.get_vocab()[mt]] - logits[idx, tokenizer.get_vocab()[wt]]).item()
        scores.append(score)
    df[f"model_{i}"] = scores
    print(f"Model {i} done")

torch.Size([883, 33])


  0%|          | 0/16777 [00:00<?, ?it/s]

Model 0 done
torch.Size([883, 33])


  0%|          | 0/16777 [00:00<?, ?it/s]

Model 1 done
torch.Size([883, 33])


...

Model 2 done
torch.Size([883, 33])


  0%|          | 0/16777 [00:00<?, ?it/s]

Model 3 done
torch.Size([883, 33])


  0%|          | 0/16777 [00:00<?, ?it/s]

Model 4 done


In [10]:
aggregate_score = 0
for i in range(0, 5):
    # min-max
    df[f"model_{i}"] = (df[f"model_{i}"] - df[f"model_{i}"].min()) / (df[f"model_{i}"].max() - df[f"model_{i}"].min())
    aggregate_score += df[f"model_{i}"]
df["aggregate"] = aggregate_score
df = df.sort_values("aggregate", ascending=False)

In [11]:
df.head(50)

,mutant,model_0,model_1,model_2,model_3,model_4,aggregate
14924,Q786L,1.000000,1.000000,1.000000,1.000000,1.000000,5.000000
8459,L446F,0.997560,0.997609,0.997531,0.997612,0.997502,4.987815
8163,S430P,0.992126,0.992140,0.992044,0.992148,0.992137,4.960597
9777,C515P,0.987867,0.987908,0.987823,0.987875,0.987881,4.939353
16425,P865L,0.983489,0.983472,0.983411,0.983487,0.983464,4.917323
15133,W797L,0.976339,0.976405,0.976350,0.976320,0.976378,4.881791
15039,R792M,0.972139,0.972199,0.972082,0.972154,0.972063,4.860637
7114,T375K,0.970342,0.970267,0.970241,0.970335,0.970324,4.851509
15932,C839N,0.970206,0.970264,0.970229,0.970216,0.970170,4.851084
2356,C125A,0.962065,0.962119,0.962033,0.962063,0.962057,4.810337
